# Практическое занятие 8
# Компьютерный практикум по алгебре на Python
## Численное решение систем линейных алгебраических уравнений (СЛАУ) с numpy.

https://numpy.org/doc/stable/reference/routines.linalg.html

In [ ]:
import numpy as np
from numpy import linalg

#Представление матриц в numpy.

!!! Сейчас матрицы в numpy рекомендуется представлять в виде структуры данных "2d numpy.array object", а не a "numpy.matrix object", даже для задач линейной алгебры.

2d numpy.array object это вложенный (двумерный) массив, его элементы (в отличие от списков list) данные одного типа, т.е. все элементы числа int, float,комплексные числа или какой-то другой числовой тип.

Сначала составим матрицу.

In [ ]:
my_matr = np.array([[1, 2, 3],
                    [4, 5, 6]])
my_matr

array([[1, 2, 3],
       [4, 5, 6]])

Если конструктору numpy.array передать список, содержащий разные типы чисел, то все числа преобразуются к наиболее общему типу. Например, если в списке есть int и float, то все станут float, а если еще есть комплексные числа, то все будут преобразованы в комплексные числа.

In [ ]:
my_matr_1 = np.array([[1, 2.5, 3],
                     [4, 5, 6]])
my_matr_2 = np.array([[1, 2 + 1j, 3],
                     [4, 5, 6]])
print('my_matr_1', my_matr_1, 'my_matr_2', my_matr_2, sep='\n')

my_matr_1
[[1.  2.5 3. ]
 [4.  5.  6. ]]
my_matr_2
[[1.+0.j 2.+1.j 3.+0.j]
 [4.+0.j 5.+0.j 6.+0.j]]


Для некоторых стандартных видов матриц есть специальные конструкторы, например

для **единичной** матрицы numpy.identity(n, dtype=None, *, like=None) и numpy.eye(N, M=None, k=0, dtype=<class 'float'>, order='C', *, like=None)

для **матрицы из нулей** numpy.zeros(shape, dtype=float, order='C', *, like=None)

In [ ]:
print(f"""np.identity(3):\n{np.identity(3)},
np.eye(3, 4):\n{np.eye(3, 4)},
np.eye(3, 4, dtype=int):\n{np.eye(3, 4, dtype=int)},
np.zeros((2, 3)):\n{np.zeros((2, 3))},
np.zeros((2, 3), dtype=complex):\n{np.zeros((2, 3), dtype=complex)}.""",
      sep='\n')

np.identity(3):
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]],
np.eye(3, 4):
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]],
np.eye(3, 4, dtype=int):
[[1 0 0 0]
 [0 1 0 0]
 [0 0 1 0]],
np.zeros((2, 3)):
[[0. 0. 0.]
 [0. 0. 0.]],
np.zeros((2, 3), dtype=complex):
[[0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j]].


## numpy.linalg.solve

linalg.solve это решатель СЛАУ из N уравнений от N переменных.Возможное число решений одно, иначе выдается ошибка.

### Пример 1. Совместная СЛАУ с единственным решением.


In [ ]:
A = np.array([[2, 3, -1],
              [3, -2, 1],
              [1, 1, -1]])
b = np.array([5, 2, 0])
X = linalg.solve(A, b)
print(f'Решение СЛАУ: {X}')

Решение СЛАУ: [1. 2. 3.]


Как выполнить проверку? В случае отсутствия ошибок округлений при вычислениях должно выполняться $AX = b$.

Для умножения матриц (т.е. array) используется оператор @ или метод matmul (то же, что и @):

In [ ]:
A @ X == b

array([ True,  True, False])

Как видим, матрицы-столбцы левой и правой частей не совпадают в одной координате. Почему? По причине округлений при вычислениях. Посмотрим, насколько сильно отличаются левая и правая части:

In [ ]:
print(f'A @ X = {A @ X},\nb = {b}')

A @ X = [5.0000000e+00 2.0000000e+00 4.4408921e-16],
b = [5 2 0]


Поскольку в жизни при использовании приближенных вычислений всегда результат подстановки решения будет несколько отличаться от правой части, то сравнение проводится лишь с некоторой точностью с помощью numpy.allclose(a, b, rtol=1e-05, atol=1e-08, equal_nan=False)

https://numpy.org/devdocs/reference/generated/numpy.allclose.html

numpy.allclose() сравнивает поэлементно первый и второй свои аргументы и возвращает True, если они все отличаются друг от друга не более чем на допустимую величину, определяемую параметрами rtol (относительная погрешность) и atol (абсолютная погрешность).

При сравнении друх чисел $a$ и $b$ вычисляется величина $|a - b|$  и сравнивается с допустимой погрешностью $atol + rtol * |b|$. Если выполняется $|a - b|\le atol + rtol * |b|$, то считается, что $a$ и $b$ приближенно равны.

In [ ]:
np.allclose(A @ X, b)

True

### Пример 2. Несовместная СЛАУ
$$
\left\{
\begin{matrix}
2x + 3y - z = 5\\
3x - 2y + z = 2\\
5x + y = 0
\end{matrix}
\right.
$$

В случае этой несовместной СЛАУ linalg.solve выдает ошибку "Singular matrix". Чтобы программа не завершалась ошибкой, будем вычислять определитель левой части (если матрица квадратная) или ранг (в общем случае).

In [ ]:
A = np.array([[2, 3, -1], [3, -2, 1], [5, 1, 0]])
b = np.array([5, 2, 0])
print(f"""Определитель |А| = {linalg.det(A)},
ранг rg(А) = {linalg.matrix_rank(A)}""")

Определитель |А| = 0.0,
ранг rg(А) = 2


Для проверки СЛАУ на совместность по теореме Кронекера-Капелли нужно определить ранг расширенной матрицы СЛАУ, а для этого надо эту матрицу составить. Для соединения двух матриц в одну в numpy есть много функций (concatenate, stack, hstack, vstack и т.п), но в нашем случае, когда нужно к двумерному массиву присоединить одномерный столбец, лучше подойдет column_stack().

In [ ]:
Ab = np.column_stack((A, b))
print(f'Ab:\n{Ab}\nранг Ab rg(Аb) = {linalg.matrix_rank(Ab)}')

Ab:
[[ 2  3 -1  5]
 [ 3 -2  1  2]
 [ 5  1  0  0]]
ранг Ab rg(Аb) = 3


Видим, что ранг расширенной матрицы больше, чем ранг левой части, следовательно, СЛАУ несовместна.

### Пример 3. Недоопределенная СЛАУ
$$
\left\{
\begin{matrix}
2x + 3y - z = 5\\
3x - 2y + z = 2\\
5x + y = 7
\end{matrix}
\right.
$$

In [ ]:
A = np.array([[2, 3, -1], [3, -2, 1], [5, 1, 0]])
b = np.array([5, 2, 7])
Ab = np.column_stack((A, b))
print(f"""ранг A rg(А) = {linalg.matrix_rank(A)},
ранг Ab rg(Аb) = {linalg.matrix_rank(Ab)}""")

ранг A rg(А) = 2,
ранг Ab rg(Аb) = 2


С помощью linalg.solve() такую СЛАУ решить нельзя, поскольку матрица левой части неполного ранга.

### Выделение строки, столбца и подматрицы в np.array
Для выделения части матрицы будем использовать диапазоны (срезы, slice)

Напомним, что обращение к элементу матрицы осуществляется указанием в квадратных скобках после имени матрицы номера строки и номера столбца через запятую,

например, $А[2, 3]$.

Если нужно выделить элементы строки, стоящие в столбцах с 3 до 5 (не включая 5!), то вместо номера столбца указываем диапазон (срез) 3:5,
двоеточие обозначает, что берутся и все промежуточные индексы.

**!!!ВАЖНО!!!**

В диапазонах не учитывается последнее значение,
так срез 3:5 не включает элемент с номером 5!

In [ ]:
Qmatr = np.array([[i * j for i in range(1, 9)] for j in range(2, 8)])
Qmatr

array([[ 2,  4,  6,  8, 10, 12, 14, 16],
       [ 3,  6,  9, 12, 15, 18, 21, 24],
       [ 4,  8, 12, 16, 20, 24, 28, 32],
       [ 5, 10, 15, 20, 25, 30, 35, 40],
       [ 6, 12, 18, 24, 30, 36, 42, 48],
       [ 7, 14, 21, 28, 35, 42, 49, 56]])

Выделим столбец с номером 1 в матрице Qmatr

In [ ]:
Qmatr[:, 1]

array([ 4,  6,  8, 10, 12, 14])

Выделим строку с номером 0 в матрице Qmatr

In [ ]:
Qmatr[0, :]

array([ 2,  4,  6,  8, 10, 12, 14, 16])

Допишем снизу к матрице Qmatr ее строку номер 1

In [ ]:
np.row_stack((Qmatr, Qmatr[1, :]))

array([[ 2,  4,  6,  8, 10, 12, 14, 16],
       [ 3,  6,  9, 12, 15, 18, 21, 24],
       [ 4,  8, 12, 16, 20, 24, 28, 32],
       [ 5, 10, 15, 20, 25, 30, 35, 40],
       [ 6, 12, 18, 24, 30, 36, 42, 48],
       [ 7, 14, 21, 28, 35, 42, 49, 56],
       [ 3,  6,  9, 12, 15, 18, 21, 24]])

Допишем справа к матрице $A$ ее столбец номер 0

In [ ]:
np.column_stack((Qmatr, Qmatr[:, 0]))

array([[ 2,  4,  6,  8, 10, 12, 14, 16,  2],
       [ 3,  6,  9, 12, 15, 18, 21, 24,  3],
       [ 4,  8, 12, 16, 20, 24, 28, 32,  4],
       [ 5, 10, 15, 20, 25, 30, 35, 40,  5],
       [ 6, 12, 18, 24, 30, 36, 42, 48,  6],
       [ 7, 14, 21, 28, 35, 42, 49, 56,  7]])

**Заметим, что np.row_stack и np.column_stack возвращают результат, не меняя саму матрицу.**

### Транспонирование
Транспонируем Qmatr с помощью transpose().

In [ ]:
np.transpose(Qmatr)

array([[ 2,  3,  4,  5,  6,  7],
       [ 4,  6,  8, 10, 12, 14],
       [ 6,  9, 12, 15, 18, 21],
       [ 8, 12, 16, 20, 24, 28],
       [10, 15, 20, 25, 30, 35],
       [12, 18, 24, 30, 36, 42],
       [14, 21, 28, 35, 42, 49],
       [16, 24, 32, 40, 48, 56]])

In [ ]:
Qmatr.T

array([[ 2,  3,  4,  5,  6,  7],
       [ 4,  6,  8, 10, 12, 14],
       [ 6,  9, 12, 15, 18, 21],
       [ 8, 12, 16, 20, 24, 28],
       [10, 15, 20, 25, 30, 35],
       [12, 18, 24, 30, 36, 42],
       [14, 21, 28, 35, 42, 49],
       [16, 24, 32, 40, 48, 56]])

## Работа с файлами.

### Чтение из файла с pandas
Данные, представленные в виде таблицы обычно хранятся в файле .xlsx или .csv.
Работать с данными, записанными в файлы такого типа, будем с помощью пакета pandas,
в нем нам понадобится функция
**pandas.read_excel**

pandas.read_excel(io, sheet_name=0, header=0, names=None, index_col=None, usecols=None, squeeze=False, dtype=None, engine=None, converters=None, true_values=None, false_values=None, skiprows=None, nrows=None, na_values=None, keep_default_na=True, na_filter=True, verbose=False, parse_dates=False, date_parser=None, thousands=None, comment=None, skipfooter=0, convert_float=True, mangle_dupe_cols=True, storage_options=None)

Подробности смотрите здесь:

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html


## Пример 1
Считать из файла 'SLAE_1.xlsx' матрицу левой части и столбец правой части и вывести их на экран.

In [ ]:
from google.colab import files
import pandas as pd
uploaded = files.upload()
for file_name in uploaded.keys():
    print(f'Загружен файл {file_name}')

Adf = pd.read_excel(file_name, sheet_name='A', header=None)
bdf = pd.read_excel(file_name, sheet_name='b', header=None)
print(Adf, bdf, sep='\n')

Saving SLAE_1.xlsx to SLAE_1.xlsx
Загружен файл SLAE_1.xlsx
          0         1         2         3         4         5         6
0  8.795979  4.875230  8.682121  4.009440  9.801702  5.501556  9.179193
1  0.531467  9.123859  7.337998  6.744377  7.362919  3.336827  7.619186
2  5.836153  7.247817  4.250346  1.218730  5.360885  1.459586  4.592442
3  6.239051  5.307652  5.385178  3.816873  5.957605  8.304191  2.498066
4  8.989077  4.345635  8.622658  3.015518  2.002419  5.449091  9.567378
5  2.617923  3.579155  8.237178  1.368073  2.715105  6.632046  8.133801
6  0.794941  6.685788  2.455254  3.599771  6.745256  8.480010  8.396094
   0
0  1
1  2
2  3
3  4
4  5
5  6
6  7


Из файла мы считали матрицу и столбец в виде DataFrame, эта структура данных содержит не только значения элементов матрицы или столбца, но и подписи строк и столбцов (index и columns). Выделим саму матрицу и столбец $b$.

In [ ]:
print(Adf.values, bdf.values, sep='\n')

[[8.79597859 4.87523014 8.68212102 4.00944011 9.80170238 5.50155636
  9.17919335]
 [0.53146707 9.12385915 7.3379975  6.74437741 7.36291863 3.33682738
  7.61918564]
 [5.83615303 7.24781653 4.25034626 1.21873004 5.36088531 1.45958594
  4.59244238]
 [6.23905113 5.30765176 5.38517778 3.81687313 5.95760469 8.30419121
  2.49806603]
 [8.98907706 4.34563516 8.62265764 3.01551823 2.00241919 5.44909105
  9.5673775 ]
 [2.61792349 3.57915529 8.23717844 1.36807276 2.71510512 6.63204596
  8.13380055]
 [0.79494132 6.68578792 2.45525413 3.59977052 6.74525583 8.48001003
  8.39609404]]
[[1]
 [2]
 [3]
 [4]
 [5]
 [6]
 [7]]


Однако вместо атрибута .values рекомендуется использовать метод .to_numpy() (см.
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_numpy.html#pandas.DataFrame.to_numpy)

Решим СЛАУ $AX=b$ и выведем полученное решение на экран

In [ ]:
A = Adf.to_numpy()
b = bdf.to_numpy()
X = linalg.solve(A, b)
print(X.T)

[[-0.05350057  0.6703254  -0.12066784 -0.60628151 -0.47324972  0.71094818
   0.26237958]]


## Запись в файл
Для записи в файл используется метод to_excel класса DataFrame из pandas.
```
df.to_excel(file_name)
```
Здесь df это DataFrame из pandas, этот объект нам нужно создать на основе наших данных, указав при необходимости надписи строк и столбцов. Матрица $A$ и столбец $b$ у нас есть в виде DataFrame, это считанные из файла Adf и bdf. Нам надо получить DataFrame из $X$.


In [ ]:
Xdf = pd.DataFrame(X)

Запишем полученное решение в формеDataFrame в файл  'SLAE_2_1.xlsx' на лист $X$.

In [ ]:
with pd.ExcelWriter(file_name, mode='a') as writer:
    Xdf.to_excel(writer, sheet_name='X', header=False, index=False)

Скачать полученный файл можно так:

In [ ]:
files.download(file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>